In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gensim.models import Word2Vec
from gensim.interfaces import CorpusABC

In [3]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [22]:
import string
import unicodedata
import re
import nltk
from nltk.corpus import stopwords
import contractions
import emoji
import demoji
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import PorterStemmer, remove_stopwords

In [5]:
df = pd.read_csv('flipkart_reviews_dataset.csv')
df

,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...,Shirala,8 months ago,1390,276
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize,Visakhapatnam,8 months ago,643,133
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...,Kozhikode,"Feb, 2020",1449,328
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i...",Jaora,7 months ago,160,28
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...,New Delhi,8 months ago,533,114
...,...,...,...,...,...,...,...,...,...
9369,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,4,Good quality product,this head phnes give good base in pluged ears ...,Bhimavaram,"Apr, 2018",0,0
9370,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,5,Awesome,This so far is one of the best headphones I've...,Ettimadai,"Apr, 2018",0,0
9371,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,4,Really Nice,I am a BASS LOVER and all others like me read ...,Guntur District,"Apr, 2018",0,0
9372,ACCEVQZABYWJHRHF,BoAt BassHeads 100 Wired Headset,5,Super!,After use of one month........ It is an excell...,Kothamangalam,"Apr, 2018",0,0


In [15]:
def preprocess(text):
    # 1. Remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # 2. Expand contractions
    text = contractions.fix(text)
    
    # 3. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # 4. Remove digits
    text = re.sub(r'\d+', '', text)
    
    # 5. Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    
    # 6. Tokenization
    tokens = word_tokenize(text)
    
    # 7. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # 8. Eliminate multiple spaces
    text = ' '.join(tokens)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 9. Perform stemming
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens]
    
    # 10. Convert to lower case
    text = text.lower()
    
    # 11. Remove emojis
    text = demoji.replace(text, '')
    
    # 12. Correct spelling
    text = str(TextBlob(text).correct())
    
    # 13. Lemmatization
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    text = ' '.join([token.lemma_ for token in doc])

In [24]:
def preprocess2(text):
    # 1. Remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # 2. Expand contractions
    text = contractions.fix(text)
    
    # 3. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # 4. Remove digits
    text = re.sub(r'\d+', '', text)
    
    # 5. Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    
    # 6. Tokenization
    tokens = word_tokenize(text)
    
    # 7. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # 8. Eliminate multiple spaces
    text = ' '.join(tokens)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [25]:
df['review']

0       1-more flexible2-bass is very high3-sound clar...
1          Super sound and good looking I like that prize
2       Very much satisfied with the device at this pr...
3       Nice headphone, bass was very good and sound i...
4       Sound quality super battery backup super quali...
                              ...                        
9369    this head phnes give good base in pluged ears ...
9370    This so far is one of the best headphones I've...
9371    I am a BASS LOVER and all others like me read ...
9372    After use of one month........ It is an excell...
9373    Excellent quality. it works so good. super hap...
Name: review, Length: 9374, dtype: object

In [26]:
from textblob import TextBlob
from bs4 import BeautifulSoup

In [29]:
df['cleaned_review'] = df['review'].apply(preprocess2)

In [30]:
tb = TextBlob("I'm concered about this issue.")
tb.correct()

TextBlob("I'm concerned about this issue.")

In [31]:
tb.ngrams(n=2)

[WordList(['I', "'m"]),
 WordList(["'m", 'concered']),
 WordList(['concered', 'about']),
 WordList(['about', 'this']),
 WordList(['this', 'issue'])]

In [32]:
sentences = [x.split() for x in df['cleaned_review']]
sentences

[['flexiblebass',
  'highsound',
  'clarity',
  'good',
  'battery',
  'back',
  'hours',
  'main',
  'thing',
  'fastest',
  'charging',
  'system',
  'available',
  'min',
  'charge',
  'get',
  'long',
  'hours',
  'back',
  'killing',
  'look',
  'awesome',
  'gaming',
  'product',
  'support',
  'want',
  'gaming',
  'recommend',
  'please',
  'buy',
  'want',
  'music',
  'product',
  'well',
  'wireless',
  'headphones',
  'comparing',
  'headphones',
  'pric'],
 ['Super', 'sound', 'good', 'looking', 'like', 'prize'],
 ['much',
  'satisfied',
  'device',
  'price',
  'point',
  'awesome',
  'brand',
  'Design',
  'wise',
  'liked',
  'rockerz',
  'modelBought',
  'model',
  'blindly',
  'first',
  'day',
  'zero',
  'reviews',
  'wonderful',
  'choice',
  'must',
  'sayBluetooth',
  'enables',
  'us',
  'connect',
  'devices',
  'simultaneously',
  'vibration',
  'motor',
  'calls',
  'good'],
 ['Nice',
  'headphone',
  'bass',
  'good',
  'sound',
  'clear',
  'purchase',
  'ca

In [35]:
wv_skipgram = Word2Vec(sentences=sentences, vector_size=100, window=5 ,sg=1)
wv_skipgram.wv.most_similar('headphones', topn=10)

[('earphones', 0.9564937949180603),
 ('comparing', 0.9369687438011169),
 ('boult', 0.9336642622947693),
 ('k', 0.9305457472801208),
 ('market', 0.9279264211654663),
 ('headsets', 0.9260472059249878),
 ('headphone', 0.9235460758209229),
 ('blindly', 0.9222476482391357),
 ('Mi', 0.9217765927314758),
 ('wired', 0.921127438545227)]

In [36]:
wv_cbow = Word2Vec(sentences=sentences, vector_size=100, window=5 ,sg=0)
wv_cbow.wv.most_similar('headphones', topn=12)

[('definitely', 0.9986463189125061),
 ('think', 0.9985246062278748),
 ('k', 0.9983253479003906),
 ('market', 0.9981566667556763),
 ('blindly', 0.9981440305709839),
 ('recommend', 0.9979444146156311),
 ('headset', 0.997944176197052),
 ('oneplus', 0.9977788329124451),
 ('say', 0.9977091550827026),
 ('Go', 0.9976367950439453),
 ('prefer', 0.9975985288619995),
 ('Bluetooth', 0.9975524544715881)]

In [37]:
df.head()

,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes,cleaned_review
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...,Shirala,8 months ago,1390,276,flexiblebass highsound clarity good battery ba...
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize,Visakhapatnam,8 months ago,643,133,Super sound good looking like prize
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...,Kozhikode,"Feb, 2020",1449,328,much satisfied device price point awesome bran...
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i...",Jaora,7 months ago,160,28,Nice headphone bass good sound clear purchase ...
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...,New Delhi,8 months ago,533,114,Sound quality super battery backup super quali...


In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [133]:
X2 = df['cleaned_review']
y2 = pd.get_dummies(df['rating'], dtype='int32').values
maxwords = 5000
maxlen = 100
tokenizer = tensorflow.keras.preprocessing.text.Tokenizer(num_words=maxwords)
tokenizer.fit_on_texts(X2)
X2_tokenizer = tokenizer.texts_to_sequences(X2)
X2_padded = pad_sequences( X2_tokenizer, maxlen=maxlen)

In [80]:
X = df['cleaned_review']
y = pd.get_dummies(df['rating'], dtype='int32').values
y 
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_tokenized, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.3,
                                                    random_state=43 )

In [81]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6561, 100), (2813, 100), (6561, 5), (2813, 5))

In [82]:
y

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=int32)

In [83]:
from tensorflow.keras.models import Sequential

In [87]:
model = Sequential()
model.add( Embedding(max_words, 128) )
model.add( SpatialDropout1D(0.2))
model.add( LSTM(100, dropout=0.2, recurrent_dropout=0.2) )
model.add( Dense(5, activation='softmax'))

In [88]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [89]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_5             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [90]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6561, 100), (6561, 5), (2813, 100), (2813, 5))

In [91]:
model.fit(X_train, y_train, batch_size=50, epochs=10, 
          validation_data=[X_test,y_test], verbose=2 )

Epoch 1/10
132/132 - 7s - 56ms/step - accuracy: 0.5900 - loss: 1.1345 - val_accuracy: 0.6321 - val_loss: 0.9884
Epoch 2/10
132/132 - 7s - 49ms/step - accuracy: 0.6709 - loss: 0.8761 - val_accuracy: 0.6619 - val_loss: 0.8986
Epoch 3/10
132/132 - 6s - 48ms/step - accuracy: 0.7029 - loss: 0.7838 - val_accuracy: 0.6644 - val_loss: 0.8910
Epoch 4/10
132/132 - 6s - 47ms/step - accuracy: 0.7353 - loss: 0.7082 - val_accuracy: 0.6466 - val_loss: 0.9133
Epoch 5/10
132/132 - 6s - 47ms/step - accuracy: 0.7595 - loss: 0.6430 - val_accuracy: 0.6584 - val_loss: 0.9530
Epoch 6/10
132/132 - 6s - 47ms/step - accuracy: 0.7773 - loss: 0.5963 - val_accuracy: 0.6580 - val_loss: 0.9820
Epoch 7/10
132/132 - 6s - 47ms/step - accuracy: 0.8006 - loss: 0.5493 - val_accuracy: 0.6541 - val_loss: 0.9939
Epoch 8/10
132/132 - 6s - 47ms/step - accuracy: 0.8188 - loss: 0.5031 - val_accuracy: 0.6363 - val_loss: 1.0816
Epoch 9/10
132/132 - 6s - 47ms/step - accuracy: 0.8322 - loss: 0.4739 - val_accuracy: 0.6406 - val_loss:

In [109]:
y_pred = model.predict(X_test)
y_pred_result = np.argmax(y_pred, axis=1)
y_pred_result

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


array([0, 4, 0, ..., 4, 0, 4])

In [110]:
y_test_result = np.argmax(y_test, axis=1)
y_test_result

array([0, 4, 0, ..., 4, 0, 4])

In [111]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

In [112]:
accuracy_score(y_test_result, y_pred_result)

0.6253110558123001

In [113]:
rpt = classification_report(y_test_result, y_pred_result)
print(rpt)

              precision    recall  f1-score   support

           0       0.70      0.73      0.71       355
           1       0.36      0.17      0.23        84
           2       0.30      0.19      0.23       194
           3       0.34      0.31      0.32       583
           4       0.73      0.79      0.76      1597

    accuracy                           0.63      2813
   macro avg       0.48      0.44      0.45      2813
weighted avg       0.60      0.63      0.61      2813



In [124]:
max_words, max_len

(5000, 100)

In [125]:
Bidirectional??

Init signature: Bidirectional(*args, **kwargs)
Source:        
@keras_export("keras.layers.Bidirectional")
class Bidirectional(Layer):
    """Bidirectional wrapper for RNNs.

    Args:
        layer: `keras.layers.RNN` instance, such as
            `keras.layers.LSTM` or `keras.layers.GRU`.
            It could also be a `keras.layers.Layer` instance
            that meets the following criteria:
            1. Be a sequence-processing layer (accepts 3D+ inputs).
            2. Have a `go_backwards`, `return_sequences` and `return_state`
            attribute (with the same semantics as for the `RNN` class).
            3. Have an `input_spec` attribute.
            4. Implement serialization via `get_config()` and `from_config()`.
            Note that the recommended way to create new RNN layers is to write a
            custom RNN cell and use it with `keras.layers.RNN`, instead of
            subclassing `keras.layers.Layer` directly.
            When `return_sequences` is `True`, 

In [127]:
bilstm_model = Sequential()
bilstm_model.add( Embedding(max_words, 128) )
bilstm_model.add( SpatialDropout1D(0.2) )
#bilstm_model.add( LSTM(max_len, dropout=0.2, recurrent_dropout=0.2))
bilstm_model.add(Bidirectional(LSTM(100, return_sequences=True)))
bilstm_model.add(Bidirectional(LSTM(100)))
bilstm_model.add(Dense(5,activation='softmax'))
                

In [128]:
bilstm_model.compile(metrics=['accuracy'],
                     loss='categorical_crossentropy', optimizer='adam')

In [129]:
bilstm_model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_9             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [131]:
bilstm_model.fit( X_train, y_train, batch_size=50, epochs=2, validation_data=[X_test, y_test] )

Epoch 1/2
132/132 ━━━━━━━━━━━━━━━━━━━━ 20s 141ms/step - accuracy: 0.5657 - loss: 1.1948 - val_accuracy: 0.6609 - val_loss: 0.9164
Epoch 2/2
132/132 ━━━━━━━━━━━━━━━━━━━━ 25s 189ms/step - accuracy: 0.6686 - loss: 0.8691 - val_accuracy: 0.6399 - val_loss: 0.9121


In [106]:
model02 = Sequential()
model02.add( Embedding(max_words, 128))
model02.add( SpatialDropout1D(0.2) )
model02.add( LSTM(max_len, dropout=0.2, recurrent_dropout=0.2) )
model02.add( Dense(5, activation='softmax'))
model02.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [107]:
model02.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_8             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [108]:
model02.fit(X_train, y_train, batch_size=50, epochs=2, 
          validation_data=[X_test,y_test], verbose=2 )

Epoch 1/2
132/132 - 9s - 66ms/step - accuracy: 0.5953 - loss: 1.1174 - val_accuracy: 0.6506 - val_loss: 0.9563
Epoch 2/2
132/132 - 8s - 60ms/step - accuracy: 0.6693 - loss: 0.8769 - val_accuracy: 0.6559 - val_loss: 0.9087


In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [115]:
d0 = 'Geeks for geeks'
d1 = 'Geeks'
d2 = 'r2j'
 
# merge documents into a single corpus
test_strings = [d0, d1, d2]
tfidfv = TfidfVectorizer()
tfidf_of_test_strings = tfidfv.fit_transform(test_strings)

In [117]:
tfidf_of_test_strings.toarray()

array([[0.54935123, 0.83559154, 0.        ],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ]])

In [119]:
print('\nWord indexes:')
print(tfidfv.vocabulary_)
 
# display tf-idf values
print('\ntf-idf value:')
print(tfidf_of_test_strings)
 
# in matrix form
print('\ntf-idf values in matrix form:')
print(tfidf_of_test_strings.toarray())


Word indexes:
{'geeks': 1, 'for': 0, 'r2j': 2}

tf-idf value:
  (0, 1)	0.8355915419449176
  (0, 0)	0.5493512310263033
  (1, 1)	1.0
  (2, 2)	1.0

tf-idf values in matrix form:
[[0.54935123 0.83559154 0.        ]
 [0.         1.         0.        ]
 [0.         0.         1.        ]]


In [120]:
cv = CountVectorizer()
cv_mat = cv.fit_transform(test_strings)

In [121]:
print(cv_mat)

  (0, 1)	2
  (0, 0)	1
  (1, 1)	1
  (2, 2)	1


In [122]:
print(cv_mat.toarray())

[[1 2 0]
 [0 1 0]
 [0 0 1]]


In [123]:
cv.vocabulary_

{'geeks': 1, 'for': 0, 'r2j': 2}